## Importe de librerías y carga del Dataset

In [ ]:
# Librerías
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

# Ruta del CSV
dataset_path = "/content/houses_medellin.csv"

# Se carga el dataset
df = pd.read_csv(dataset_path)

# Se hizo un ajuste opcional para mejor manejo del Dataset
df.rename(columns={df.columns[0]: 'Casa #'}, inplace=True)



## Limpieza de valores númericos

In [ ]:
# Se limpian todos los valores númericos
def clean_numeric(value):
    if isinstance(value, str):
        value = value.replace("$", "").replace("COP", "").replace(",", "").replace("*m²", "").strip()
        try:
            return float(value)
        except ValueError:
            return np.nan
    return value

# Y se aplica a columnas que sean más relevantes
numeric_columns = ["Precio", "Área construída (m²)", "Área privada (m²)", "Administración", "Precio m²"]
for col in numeric_columns:
    df[col] = df[col].apply(clean_numeric)



## Modificación y calculo de columnas

In [ ]:
# Calculo del precio
df["Precio m²"] = df["Precio"] / df["Área construída (m²)"]

# Se modifican los valores de la columna antiguedad para que sea solo números
df["Antigüedad"] = df["Antigüedad"].replace({"1 a 8 años": 4.5, "9 a 15 años": 12, "16 a 30 años": 23, "+30 años": 35})
df["Antigüedad"] = pd.to_numeric(df["Antigüedad"], errors="coerce")

# Eliminación de valores nulos
df_clean = df.dropna(subset=["Precio m²", "Área construída (m²)", "Estrato"])

# Caracteristicas relevantes
features = ["Precio m²", "Estrato", "Área construída (m²)"]
df_model = df_clean[features].dropna()



## Aplicación de Isolation Forest para detectar datos anómalos

In [ ]:
# Se aplica Isolation Forest para detectar anómalias
iso_forest = IsolationForest(contamination=0.05, random_state=42)
df_model["Anomaly"] = iso_forest.fit_predict(df_model)

# Opcionalmente se agrega otra columna para identificar anómalias
df_clean["Anomaly"] = df_model["Anomaly"]

# Se separan los datos en normales y anómalos
df_anomalies = df_clean[df_clean["Anomaly"] == -1]
df_corrected = df_clean[df_clean["Anomaly"] == 1]



## Guardado e impresión de resultados (en .CSV)

In [3]:
# Los datos se guardan en archivos CSV
df_anomalies.to_csv("/content/datos_atipicos.csv", index=False)
df_corrected.to_csv("/content/datos_corregidos.csv", index=False)

# Los resultados que se muestran son los datos atípicos que se identificaron
print(f"Total de datos: {len(df_clean)}")
print(f"Datos atípicos detectados: {len(df_anomalies)}")

Total de datos: 274
Datos atípicos detectados: 14
